# CSPB 3104 Assignment 10:
## Instructions

> This assignment is to be completed and uploaded to 
moodle as a python3 notebook. 

> Submission deadlines are posted on moodle. 

> The questions  provided  below will ask you to either write code or 
write answers in the form of markdown.

> Markdown syntax guide is here: [click here](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet)

> Using markdown you can typeset formulae using latex.

> This way you can write nice readable answers with formulae like thus:

>> The algorithm runs in time $\Theta\left(n^{2.1\log_2(\log_2( n \log^*(n)))}\right)$, 
wherein $\log^*(n)$ is the inverse _Ackerman_ function.

__Double click anywhere on this box to find out how your instructor typeset it. Press Shift+Enter to go back.__


## Question 1: Arbitrage Opportunities

An arbitrage in finance is a way to make money for sure without much risk. For instance, suppose 1 USD yields 5 Chinese RMB and 1 RMB trades for 0.4 Euros, and 1 Euro trades for 0.52 USD,  you have an arbitrage opportunity
wherein you can take x USD, obtain 5x RMB, and in turn trade them for 2x Euros and finally end up with 1.02x USD back. Every time you exercise this opportunity, you get a 2 cent profit (assuming you can trade for free and there is no spread in the buying and selling prices, but that is another can of worms).

In theory, arbitrage cannot exist in efficient markets but they can persist for a short amount of time. Electronic traders can then search for such opportunities in real time and rapidly exploit them.

You are given a basket of $n$ currencies and for each pair of currencies $(C_i,C_j)$, you are given a ratio
$R_{i,j}$ that says that one unit of currency $C_i$ is currently fetching $R_{i,j}$ units of $C_j$.

__(A)__ Given the ratios $R_{i,j}$ write an efficient algorithm that can detect if any arbitrage opportunities currently exist. What is the running time of your algorithm?

(__Hint:__ Make a graph out of the problem data. What does the presence of arbitrage mean for this graph? )

__(B)__ We are not just interested in finding an arbitrage opportunity, but also interested in finding the shortest length opportunity, defined as one that involves the smallest number of trades. Naturally, given that prices change
and also given that there are trading fees, such opportunities are more desirable. 

Write an algorithm to find the smallest length arbitrage opportunity. What is the running time for your algorithm?

(__Hint:__ Modify the Floyd Warshall algorithm you learned this week. In particular, how do you detect if there is a negative weight cycle of length k in the graph?)




### Answer 1 (Expected Length: 12 lines).

__(A)__  
First, build a directed graph where each node represents a currency type. A directed edge from node $C_i$ to $C_j$ means that currency $C_i$ can be exchanged for currency $C_j$ by an exchange rate of the weighted edge $R_{i,j}$. An edge should also be built from $C_j$ to $C_i$ with an edge weight of $1/R_{i,j}$. 

The arbitage problem is about finding the maximum amount of currency that can be gained through exchanges, but shortest path algorithms find the shortest path. Thus, we need to modify our graph slightly. Exchange rates are multiplicative, so first make the edge weights the log of the original edge weights.  Next, negate all the edge weights, and build the graph as described above.

An arbitage opportunity can then be represented by a cycle between nodes, with an overall negative weight, representing a gain in currency through trading (and a positive cycle in the original problem). To find the a cycle/arbitage opportunity, run the standard Floyd Warshall algorithm on the negated graph, to find shortest path distances in a 2d matrix. If any currencies have a path that connects the node to itself (in a loop), this means that an arbitage opportunity involving that currency exists. This will appear as a negative value in the matrix on the diagonal, where a node has a cycle to itself. 

The key is this: A negative-weight cycle on the negative-log graph represents an arbitrage opportunity.  

Floyd-Warshall's complexity is $\Theta(V^3)$. 

__(B)__
We are tasked with finding the shortest cycle in the original graph from 1A). To do this, we can use a slightly modified Floyd-Warshall algorithm. Set the $distance[u][V]$ to be infinity, as normal. That is, the distance from a given node, u to any other vertex is initialized to infinity. The modified part is that $distance[u][u]$   is initialized to infinity, instead of 0. The algorithm will then work to find the shortest distance from a given node, u, to itself in a cycle. Finally, we find the minimum of all values of $distance[u][u]$  , where u is any vertex that connects to itself in a loop. This minimum value will be the smallest length arbitage opportunity. 

Stated another way, to detect negative cycles after running FW, look at the diagonal values of the 2d matrix, where $i==j$. A negative diagonal value (in the negative-log graph) means there is a negative loop from the vertex to itself. Find the minimum of all such values to find the smallest length arbitrage opportunity.

This has the same complexity of Floyd-Warshall's algorithm, $\Theta(V^3)$.


----

### Question 2: Longest Paths.

Given a weighted, directed graph $G$, the longest simple path from vertex $s$ to $t$ is a path that (a) goes from $s$ to $t$, (b) cannot revisit a vertex along the path, and (c) has the maximum weight among all the paths going from $s$ to $t$.

__(A)__ Show using an example that for any graph $G$, that the longest path problem is not equivalent to solving the shortest path problem by negating the edge weights. (*Hint* Negating edge weights will work but for a common problem that makes shortest paths undefined.)

__(B)__ Show that if the graph $G$ is a DAG, the longest path problem can be solved by negating the edge weights and solving a shortest path problem. What is the running time cost?


### Answer 2 (Expected Length: 6 lines)

__(A)__ 
Negating the edge weights then solving the shortest path problems will not always work. If there are no negative weight cycles, then the shortest distances from the source are all well defined. If we have a graph that has positive weight cycles, when we negate weights, this will lead to a graph with a negative weight cycle, which makes the shortest distances from s undefined. 

__(B)__ 
If a G is a DAG, then negating the edge weights then solving the shortest path problem is viable. This is because a DAG, by definition, has no cycles, and thus the negation will have no negative cycles. 

We can solve this problem by doing a modified shortest path algorithm, as described in lecture. This is equivalent to doing one round of a (modified) Bellman-Ford iteration.  

First, negate the edge weights. Next, run DFS to form a topological sort, such that the vertices appear in descending order based on finish time (or from lower rank to higher rank).   
Iterate through vertices in the order of the topological sort, and for each outgoing edge, relax that edge distance if possible.  

The complexity is $\Theta(|V|+|E|)$.  



----

### Question 3: Being In the Right Place At the Right Time.

You are helping a secret agent plan a series of rendezvous in a capital. The rendezvous are happening along different stations of a subway line at precisely hours $1, 2, \ldots, n$.  There are $m$ subway stations, each with an ID numbered $1, \ldots, m$.
The rendezvous at hour $j$ occurs at station $S_j$:

$$\begin{array}{|l|c|c|c|c|c|}
\hline
\text{Rendezvous} & 1 & 2 & \cdots & n-1 & n \\
\hline
\text{Station ID} & S_1 & S_2 & \cdots & S_{n-1} & S_n \\
\hline \end{array}$$

For instance, The rendezvous at hour 1 might happen at station 5, the rendezvous at hour 2 might happen at station 12, and the rendezvous at hour 3 might happen at station 1.  In this case, $S_1 = 5, S_2 = 12,$ and $S_3 = 1$.  __Note:__ Your answer should be for any sequence of rendezvous, not for this specific example.

The spy starts at station $1$ at time $0$ and further, must at all costs attend rendezvous $n$. In each hour, the agent is limited to travelling no more than $K$ stations along the line.  Thus, she must decide whether to skip some of the rendezvous in favor of others.

Provide an algorithm to calculate the maximum number of rendezvous the agent can make with the constraint that she must make rendezvous $n$. What is the running time?

__Hint:__ As usual, it comes down to defining a graph and solving a suitable problem.


### Answer 3 (expected length: 6 lines)

This problem can be represented as a directed, acyclic graph connecting stations $S_1$...$S_n$ along a line. 

We know that the person can travel no more than K stations along the line at a given hour, so that they can only travel to a station<=K stops away. We also know that the person must be at station $S_n$ for rendezvous n. Furthermore, to make a meeting at a station $S_j$ they must be at that station by hour j. 

To solve this problem, create a directed acyclic graph connecting the stations along a line, where each edge connecting two nodes (stops) has a weight of 1. There will be a reverse edge of cost 1 if the train can move the opposite direction, back to the previous stop. Every node should have two identifiers, $j$ to note the rendezvous hour and $S_j$ to note the station id. Unconnected nodes have an infinity edge weight.  

Make a list, sorting the nodes by $j$, the hour at which the rendezvous could happen. Next, run Dijkstra's algorithm on each of these in order from the lowest to highest rendezvous times. This will find the shortest paths from each node to every other reachable point, including the end point, $S_n$. 

Afterwards, create a new graph representing all paths from each source node $S_j$ to the destination, $S_n$, if the number of edges from source to $S_n$ is $<=k$. Each edge should have a weight of -1, because we seek to find the longest path (most number of stops) rather than the shortest path. 

Finally, find the shortest distance from the source node, $S_1$ to $S_n$ in this new graph using Bellman-Ford's algorithm. This will find the shortest distance to the destination (the "longest" negative distance), which represents the maximum number of stops that can be made in the original problem.   

Complexity analysis: 
N iterations of Dijkistra:$\Theta(N*(|E|+|V|)log|V|)$.  
1 iteration of BellmanFord: $\Theta (|V|*|E|)$.  
Overall complexity:$\Theta(|V|*|E|+N*(|E|+|V|)log|V|)$.






----

## Question 4: U-turn if you want to.

A $N \times N$ maze is defined by a graph with vertices $(i,j)$ for $1 \leq i \leq n$ and $1 \leq j \leq n$.
Each node $(i,j)$ is connected to a subset of its four adjacent neighbors $\{(i+1, j), (i, j+1), (i-1, j), (i, j-1) \}$.

The grid is laid out so that $(1,1)$ is the south west corner and $(n,n)$ is the northeast corner.

The vehicle starts at $(1,1)$ *oriented north* and needs to reach $(n,n)$ *oriented east*.

It can travel along the four cardinal directions $N, E, W, S$ and rapidly change these direction by 
making a left or right turn. Eg., changing heading from $N$ to $E$ 
requires making a right turn, $N$ to $W$ requires a left turn and $N$ to $S$
requires either 2 left or 2 right turns.

Find the minimimum cost path from $(1,1)$ oriented north to $(n,n)$ oriented east, 
where the cost is defined as the number of 
edges plus $2 \times$ the number of left turns plus $3 \times$ the number of right turns.

What is the running time of your algorithm?

*Hint* Define a new graph that helps us track not just the vehicle location but also its current travel direction.
Define edge weights appropriately so that the shortest cost problem on this new graph will solve the original problem.

### Answer 4 (Expected Length: 8 lines)
The simplest solution is to build a weighted, directed graph by making four nodes for every position $(i,j)$, one for each orientation in the maze. Each node has four outgoing edges connected to four other nodes, where three of the edges represent turning from the current orientation, remaining at position $(i,j)$. The fourth edge represents travelling along the current bearing to a different $(i,j)$ position, if possible.  

There will be edges with specific weights between every node and four of its adjacent neighbors: the ones to the front (N), back (S), left (W) and right (E) of the current bearing.   

The edge leading to the node in the front (relative N) will either have a weight of 1 or infinity, representing travelling along the current bearing to a different $(i,j)$ position. If a position $(i,j)$ at a current bearing (one of N,E,S,W) is connected to a position $(i2,j2)$, then the weight of the "front" edge connecting the distinct positions will be 1, else it will be infinity.   

The other three edges represent making turns, but staying at the same $(i,j)$ position.  
The edge to the node to the right (relative E) will have weight 3, representing the cost of turning right from the current bearing.  
The edge to the node to the left (relative W) will have a weight of 2, representing the cost of turning left from the current bearing. The edge to the node to the back (relative S) will have a weight of 4, representing the cost of turning left twice from the current bearing, the minimum cost to turn left. 


Finally, run Dijkstra's algorithm from $(1,1)$, which will find the shortest path distance to $(n,n)$. This models the costs of 2 for turning left, 3 for turning right, 4 for turning downwards, and 1 for moving to a different $(i,j)$ position. 

For each position there will be 4 nodes and 16 edges. The complexity of building the graph is then $\Theta(4*|E|+16*|V|)$. 
The running time of Dijkstra's algorithm algorithm using a min-heap as a priority queue is $\Theta((|E|+|V|)log|V|)$.  This will be the leading term, and the overall complexity.  





----

## Testing your solutions -- Do not edit code beyond this point